# Рекуррентные нейросети

Построим простейшую нейросеть для посимвольной генерации текста

In [4]:
import pandas as pd  # для работы с данными
import time  # для оценки времени
import torch  # для написания нейросети
import re
import random as r
import os

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/DOM')

## Загрузка данных

Будем работать с датасетом реплик из Симпсонов. Нам нужно извлечь предобработанные тексты и закодировать их числами

In [6]:
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [7]:
phrases = df['normalized_text'].tolist()  # колонка с предобработанными текстами
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [8]:
text = [[c for c in ph] for ph in phrases if type(ph) is str] # проверяем, что нет NaN или числовых значений,
                                                              # они будут удалены (наверно в DataSet было бы проще сделпть)

In [9]:
len(phrases), len(text)

(11639, 10891)

## Создаём массив с данными

Нужно

1. Разбить данные на токены (у нас символы)
2. Закодировать числами
3. Превратить в эмбеддинги

In [10]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')  # все символы, которые мы хотим использовать для кодировки = наш словарь
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]  # все неизвестные символы будут получать тег none
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}  # словарь токен-индекс

In [13]:
print(CHARS)

{'w', 'u', 'g', 'b', 'y', 'x', 'q', 't', 'k', 'i', 'd', 'r', ' ', 'z', 'l', 'n', 'p', 'j', 'f', 'h', 'v', 'o', 'c', 's', 'm', 'e', 'a'}


In [15]:
print(INDEX_TO_CHAR)

['none', 'w', 'u', 'g', 'b', 'y', 'x', 'q', 't', 'k', 'i', 'd', 'r', ' ', 'z', 'l', 'n', 'p', 'j', 'f', 'h', 'v', 'o', 'c', 's', 'm', 'e', 'a']


In [17]:
print(CHAR_TO_INDEX)

{'none': 0, 'w': 1, 'u': 2, 'g': 3, 'b': 4, 'y': 5, 'x': 6, 'q': 7, 't': 8, 'k': 9, 'i': 10, 'd': 11, 'r': 12, ' ': 13, 'z': 14, 'l': 15, 'n': 16, 'p': 17, 'j': 18, 'f': 19, 'h': 20, 'v': 21, 'o': 22, 'c': 23, 's': 24, 'm': 25, 'e': 26, 'a': 27}


In [ ]:
len(INDEX_TO_CHAR)

28

In [19]:
X.shape # нулевая матрица, в которой строки - это число фраз в списке text, а столбцы - это батч, здесь = 50

torch.Size([10891, 50])

In [22]:
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода
X = torch.zeros((len(text), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
X
for i in range(len(text)):  # для каждого предложения
    for j, w in enumerate(text[i]):  # для каждого токена (токен здесь - буква или символ (единичный))
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [23]:
X[0:2]

tensor([[25, 27,  3,  3, 10, 26, 13, 15, 22, 22,  9, 13,  1, 20, 27,  8, 24, 13,
          8, 20, 27,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [15, 26, 26,  0, 25,  2, 12, 13, 15, 26, 26,  0, 25,  2, 12,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

## Embedding и RNN ячейки

Каждому токену мы хотим сопоставить не просто число, но вектор. Поэтому вектор текста нам нужно умножить на матрицу эмбеддингов, которая тоже будет учиться в процессе обучения нейросети. Для создания такой матрицы нам нужен слой `nn.Embedding`

In [24]:
X[0:5].shape

torch.Size([5, 50])

In [25]:
embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)  # размер словаря * размер вектора для кодировки каждого слова
t = embeddings(X[0:5])
t.shape

torch.Size([5, 50, 28])

In [26]:
t.shape, X[0:5].shape

(torch.Size([5, 50, 28]), torch.Size([5, 50]))

In [29]:
rnn = torch.nn.RNN(28, 128, batch_first=True)  # на вход - размер эмбеддинга, размер скрытого состояния и порядок размерностей
# скрытое состояние - это на сколько должна быть большой матрица контекста
o, s = rnn(t) # o - выход из последнего слоя модели, s - последнее скрытое состояние
# вектора для слов: батч * число токенов * размер скрытого состояния
# вектор скрытого состояния: число вектров (один) * батч * размер скрытого состояния
o.shape, s.shape

(torch.Size([5, 50, 128]), torch.Size([1, 5, 128]))

Можно применять несколько рекуррентных ячеек подряд

In [30]:
o, s2 = rnn(t, s)
o.shape, s2.shape

(torch.Size([5, 50, 128]), torch.Size([1, 5, 128]))

## Реализация сети с RNN
3 слоя:
1. Embeding (30)
2. RNN (hidden_dim=128)
3. Полносвязный слой для предсказания буквы (28, то есть размер словаря)

In [31]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(28, 30)
        self.rnn = torch.nn.RNN(30, 128)
        self.out = torch.nn.Linear(128, 28)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x, s = self.rnn(x) # берём выход с последнего слоя для всех токенов, а не скрытое состояние
        return self.out(x)

In [32]:
model = Network()

In [33]:
criterion = torch.nn.CrossEntropyLoss()  # типичный лосс многоклассовой классификации
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

Обучение:

In [34]:
for ep in range(20):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        # берём батч в 100 элементов
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 5.144, Train loss: 2.113
Epoch 1. Time: 3.671, Train loss: 1.859
Epoch 2. Time: 3.709, Train loss: 1.804
Epoch 3. Time: 4.591, Train loss: 1.772
Epoch 4. Time: 4.493, Train loss: 1.750
Epoch 5. Time: 3.641, Train loss: 1.733
Epoch 6. Time: 3.690, Train loss: 1.720
Epoch 7. Time: 5.060, Train loss: 1.709
Epoch 8. Time: 3.973, Train loss: 1.700
Epoch 9. Time: 3.664, Train loss: 1.693
Epoch 10. Time: 3.903, Train loss: 1.687
Epoch 11. Time: 5.131, Train loss: 1.682
Epoch 12. Time: 3.683, Train loss: 1.677
Epoch 13. Time: 3.624, Train loss: 1.674
Epoch 14. Time: 4.470, Train loss: 1.670
Epoch 15. Time: 4.454, Train loss: 1.667
Epoch 16. Time: 4.684, Train loss: 1.665
Epoch 17. Time: 4.177, Train loss: 1.662
Epoch 18. Time: 5.081, Train loss: 1.660
Epoch 19. Time: 3.658, Train loss: 1.658


## Генерация


- Сначала отправлем в модель буквы из предложения (прогревая состояние)
- Затем берём самую вероятную букву и добавляем её в предложение
- Повторяем пока не получим none (0)

In [35]:
CHAR_TO_INDEX['none']

0

In [36]:
def generate_sentence(word):
    sentence = list(word)
    sentence = [CHAR_TO_INDEX.get(s, 0) for s in sentence]
    answers = model.forward(torch.tensor(sentence))
    probas, indices = answers.topk(1)
    return ''.join([INDEX_TO_CHAR[ind.item()] for ind in indices.flatten()])

In [37]:
generate_sentence('dog')

' u '

In [38]:
generate_sentence('It is')

'nonehtn '